## Import Packages

In [1]:
import numpy as np
import nexfile # a .py file
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Optimizer
import matplotlib.pyplot as plt
import copy
import scipy
import scipy.io
import scipy.linalg
import scipy.stats

## Data Loading Functions

In [2]:
def loadnpz(name, allow_pickle=False):
    """
    loadnpz loads compressed files
    Args:
       name (str): directory of npz file
       allow_pickle (bool): argument to allow pickle
    Returns:
       data (np array): np array from compressed npz file
    """
    if allow_pickle:
        data = np.load(name, allow_pickle=True)  # Over-rule default False (loading pickled data can execute arbitrary code)
    else:
        data = np.load(name)
    data = data.f.arr_0  # Gets np array from data, which is currently an instance of class NpzFile, which has f attribute (numpy.lib.npyio.NpzFile)
    return data

def loadFileNames(data_dir):
    """
    Gives the names of the files in the folder folderName1, which is simply the numeric data folders from /Rodent WFU DNMS
    Args:
        data_dir (str): local directory that stores the folder for neuron data
    Returns:
        fileNames (list): list of all .nex filenames as full directory name as str (./Rodent_WFU_DNMS/1193/1193u044merge-clean.nex)
    """
    dir_name =  data_dir + '/driveNeuron' # Local directory as string for Rodent WFU DNMS
    folderNames = os.listdir(dir_name) # Get list for directory contents
    rat_nums = [] # List to append folders with integer name as rats' id
    
    # If subdirectory in list is integer then add to list rat_nums
    for num in folderNames: 
        try:
            int(num)
            rat_nums.append(num)
        except:
            True    
    
    fileNames = [] # the full directory of each event for all the rats
    
    # Generate list of all WFU Rat DMS file names with full directory name
    for num in rat_nums: 
        folder = dir_name + "/" + num + "/" # folder directory for each rat
        events = os.listdir(folder) # list of one rat's .nex file name for each event
        for a in range(len(events)): 
            events[a] = folder + events[a] # concatenate the full directory name for each event .nex file
        fileNames = fileNames + events 

    return fileNames


In [3]:
# ***Change this according to local directory***
data_dir = '/Volumes/TOSHIBA/data'

## Model Classes Defined

In [4]:
class ConstantModel(nn.Module):
    """
    ConstantModel class initializes the Z matrix for the latent dimension representation of the difference between
    datasets that is used after the embedding matrices are already training and held constant.
    Args:
        numData (int): number of data examples in design matrix
        codeSize (int): dimension of z, the latent dimension representation of the difference between datasets
    Returns:
        code (Tensor, dim=[1,z]): z for the dataKeys passed as argument
    """
    def __init__(self, numData, codeSize):
        super(ConstantModel, self).__init__()
        self.codeSize = codeSize
        self.numData = numData  # print (self.numData)
        self.code = nn.Parameter(torch.FloatTensor(np.random.random((numData, codeSize)) - 0.5))    # Random initialization of Z for all data examples

    def forward(self, dataKeys):
        code = self.code
        code = code[dataKeys]   # Code reset to Z for the dataKeys passed as argument: dim=(1,Z)
        return code

In [5]:
class SimpleNet3(nn.Module):
    """
    SimpleNet3 class generates Stefan's results for rat data and for simulation data but NOT for human data
    Args:
        codeSize (int): dimension of z, the latent dimension representation of the difference between datasets
        wireNum (np array (vector), dim=[1,num_recorded_neurons]): np array of wire index of each recorded neuron
        timeInt (int): total time before and after the event in seconds
        numEvents (int): number of events predicted
    Returns:
        code (Tensor, dim=(1,z)): z for the dataKeys passed as argument
    Special:
    Stefan commented-out hyper-parameter attempts were codeSize: {10,>10}, codeNeuron:{4,20}, N1:{10,20}, N2:{20,30},
    codeN1: {20,40,400}, f1 fit length: {5,10,20}
    N2 appears hard coded but never used so was removed by Bryan
    """
    def __init__(self, codeSize, wireNum, timeInt, numEvents):
        super(SimpleNet3, self).__init__()
        self.originalNeuron = wireNum.shape[0]  # Recorded neurons as length of np array vector of wires by their indices, int
        self.codeNeuron = 20    # Logical neurons (latent space representation of recorded neurons; M in v1 model fig), int
        # N1 = 20 # This is what Stefan calls M in his figures, the number of logical neurons (the same as codeNeuron?)

        self.filter1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(1, 20), stride=(1, 20))   # f1 layer, output dim = (batch,20,M,0.05*T)
        self.filter2 = nn.Conv2d(in_channels=20, out_channels=5, kernel_size=(1, 1), stride=(1, 1))     # f2 layer, output dim = (batch,20,M,0.05*T)

        numWire = np.unique(wireNum).shape[0]   # Number of unique wires from this recording
        cellMatrix = torch.zeros((wireNum.shape[0], numWire))   # Create all zero tensor, dim=(num_recorded_neurons,num_unique_wires), tensor
        cellMatrix[np.arange(wireNum.shape[0]), wireNum] = 1    # Double indexing to create one-hot-matrix for each recorded neuron's respective wire index, tensor
        self.cellMatrix = cellMatrix
        self.numWire = numWire
        self.cellChannel = 2    # Hard-coded number of "cells" in Stefan's nomenclature. This is NOT an actual torch channel. See Fig v1.

        codeN1 = 20    # Dimension of output from first fully connected layer in linear layers from Z_s to E_s in Fig v1
        self.codeN1 = codeN1
        self.linCode0 = torch.nn.Linear(codeSize, codeN1)  # First fully connected layer Z_s to intermediate for eventual E2, FC1 in Fig v1
        self.linCode1 = torch.nn.Linear(codeN1, self.numWire * self.codeNeuron * self.cellChannel)  # Second fully connected layer intermediate to E2, FC2 in Fig v1
        self.linCell0 = torch.nn.Linear(codeSize, codeN1)  # First fully connected layer Z_s to intermediate again for eventual E1
        self.linCell1 = torch.nn.Linear(codeN1, self.originalNeuron * self.cellChannel)  # Second fully connected layer intermediate to E1
        self.lin1 = torch.nn.Linear(self.codeNeuron * int(25 / 5 * (timeInt/2)), numEvents)  # Third fully connected layer, FC3 in Fig v1 (N1 to self.codeNeuron)
        self.nonlin = torch.tanh  # Nonlinear activation function

    def forward(self, x, z, returnEncode=False, runInput=True):
        # Args: x = N neurons x T timesteps training example, z = codes (codeModel + noise)
        z1 = self.linCell0(z)
        z1 = self.nonlin(z1)
        cellEncode = self.linCell1(z1)  # Output is E1, called cellEncode, dim=(W*2*4)
        cellEncode = (torch.tanh(cellEncode) + 1) / 2   # Map cellEncode to 0-1

        z2 = self.linCode0(z)
        z2 = self.nonlin(z2)
        encodeNeuron = self.linCode1(z2)    # Output is E2, called encodeNeuron
        encodeNeuron = encodeNeuron.reshape((z.shape[0], self.numWire * self.cellChannel, self.codeNeuron))    # Reshape from (batch,2WM) to (batch,2W,M)
        encodeNeuron = torch.softmax(encodeNeuron, axis=1)  # Softmax so each row sums to 1 (each neron is now a weighted sum to 1 of new logical neurons)

        if runInput:
            cellEncode = cellEncode.repeat_interleave(x.shape[2])  # dim=(batch,W*2*4)=(batch,8W) to dim=(batch,T8W) ?
            cellEncode = cellEncode.reshape((x.shape[0], x.shape[1], self.cellChannel, x.shape[2]))  # (batch,T8W)=(batch,T*2*4W)=(batch,T*2*N) to (batch,N,2,T)
            x = x.reshape((x.shape[0], x.shape[1], 1, x.shape[2]))  # (batch,N,T) to (batch,N,1,T)
            x = x * cellEncode  # Output dim = dim(cellEncode) = (batch,N,2,T)
            x = x.reshape((x.shape[0], x.shape[1], x.shape[2] * x.shape[3]))  # (batch,N,2,T) to (batch,N,2T)
            x = torch.swapaxes(x, 1, 2)  # (batch,N,2T) to (batch,2T,N)
            x = torch.matmul(x, self.cellMatrix)  # Output dim = (batch,2T,num_unique_wires)
            x = x.reshape((x.shape[0], self.cellChannel, x.shape[1] // self.cellChannel, x.shape[2]))  # (batch,2T,W) to (batch,2,T,W)
            x = torch.swapaxes(x, 1, 2)  # (batch,2,T,num_unique_wires) to (batch,T,2,W)
            x = x.reshape((x.shape[0], x.shape[1], x.shape[2] * x.shape[3]))  # (batch,T,2,W) to (batch,T,2W)
            x = torch.matmul(x, encodeNeuron)  # Output dim = (batch,T,M)
            x = torch.swapaxes(x, 1, 2)  # (batch,T,M) to (batch,M,T)
            x = x.reshape((x.shape[0], 1, x.shape[1], x.shape[2]))  # (batch,M,T) to (batch,1,M,T)
            x = self.filter1(x)  # Output dim = (batch,20,M,0.05*T)
            x = self.nonlin(x)
            x = self.filter2(x)  # Output dim = (batch,5,M,0.05*T)
            x = self.nonlin(x)
            x = x.reshape((x.shape[0], x.shape[1] * x.shape[2], x.shape[3]))  # (batch,5,M,0.05*T) to (batch,5M,0.05*T)
            x = x.reshape((x.shape[0], x.shape[1] * x.shape[2]))  # (batch,5M,0.05*T) to (batch,0.25*M*T)
            x = self.lin1(x)  # Output dim = (batch,6)

        if returnEncode:
            encodeNeuron = encodeNeuron.reshape((encodeNeuron.shape[0], 2, encodeNeuron.shape[1] // 2, encodeNeuron.shape[2]))  # (batch,2W,M) to (batch,2,W,M)
            encodeNeuron = torch.transpose(encodeNeuron, 1, 2)  # (batch,2,W,M) to (batch,W,2,M)
            encodeNeuron = encodeNeuron.reshape((encodeNeuron.shape[0], encodeNeuron.shape[1] * encodeNeuron.shape[2], encodeNeuron.shape[3]))  # (batch,W,2,M) to (batch,2W,M)
            return x, encodeNeuron  # returns same shape as initial input encodeNeuron but rows are not in same original order
        else:
            return x

## Model Training

In [6]:
def trainModel(data_dir, fileLabel, otherLabel = "", XtimeInt = np.arange(100), codeSize = 20, codeNorm = 100, Rat_rep = False):
    """
    This function trains the model to predict output events of Rat behavior using input neuron trains around the event
    Saves training results:
        session accuracy,
        confusion matrix,
        predictions,
        train accuracy over each iteration,
        test accuracy over each iteration
    Args:
        data_dir (str): local directory that stores the folder for neuron data
        fileLabel (str): label for saving files
        otherLabel (str): additional label for saving files
        XtimeInt (array of int): index for extracting interested time interval around the event
        codeSize (int): hyperparameter controls the size of the latent representation  
        codeNorm (int): hyperparameter that affects the regularization of the latent representation
        Rat_rep (Booleans): controls training with session or rat representation 
    Returns:
        None
    """
    # Loading Rat Data
    wireName = loadnpz(data_dir + '/eventData/general/wireNames.npz')
    X = loadnpz(data_dir + '/eventData/combined/data_' + fileLabel + '.npz') # (x, y, z) x = total events, y = neurons, z = timeframe
    Y = loadnpz(data_dir + '/eventData/combined/outputType_' + fileLabel + '.npz')
    dataKeys = loadnpz(data_dir + '/eventData/combined/keys_' + fileLabel + '.npz') # session label for each event
    
    # If "disableCodes" is True, then the model treats all sessions equally (NO TRANSFER LEARNING)
    disableCodes = False 
    if disableCodes:
        dataKeys = dataKeys * 0

#     locNames = loadnpz('./data/eventData/general/brainLocationNames.npz')
#     X[:, locNames==1] = 0

    # Edit time interval being used for prediction
    X = X[:, :, XtimeInt]
    
    # change this to True? when need to replace session specific representations with rat specific representations
    if Rat_rep:
        validArgs = loadnpz(data_dir + '/eventData/seperate/validArgs_' + fileLabel + '.npz') # len = 519
        fileNames = loadFileNames(data_dir) # len = 520
        fileNames = np.array(fileNames)
#         fileNames = fileNames[validArgs] # len = 519

        rat_idx = [] # folderNames
        for a in range(len(fileNames)):
            name = fileNames[a]
            name = name.split('/')
            rat_index = [val for val in name if val.isdigit()]
            rat_idx.append(rat_index)
            
        rat_idx = np.array(rat_idx)
        # unique1 = The sorted unique values. 
        # index1 = The indices to reconstruct the original array from the unique array. 
        # rat_idx = The indices of the first occurrences of the unique values in the original array.
        unique1, index1, rat_idx = np.unique(rat_idx, return_inverse=True, return_index=True)
        dataKeys = rat_idx[dataKeys] # rat index label for each event
        
        fileLabel = "Rat" + fileLabel 
        print(fileLabel)

    numDataSet = int(np.max(dataKeys) + 1) # number of representations
    loss_func = nn.CrossEntropyLoss()

    # "codeSize" is the size of the representation for the data set.
#     codeSize = 10
#     codeSize = 20 # for per rat rep
#     codeSize = 40
#     codeSize = 60
#     codeSize = 80

    # "noiseScale" determines the amount of noise which is added to the representation
    noiseScale = 0.01

    # "codeNorm" is the scale of the L2 normalization applied to the representation
#     codeNorm = 0.2
#     codeNorm = 0.5#0.2#0.1#0.05#
#     codeNorm = 1.0
#     codeNorm = 2.0
#     codeNorm = 4.0 #for per rat rep
#     codeNorm = 10.0
#     codeNorm = 50.0
#     codeNorm = 100.0 # seems good
    batchSize = 32

#     sensorLocation = torch.tensor(sensorLocation).float()

    Ndata = X.shape[0] # Ndata = originalSize
    Ntest = Ndata // 5
    Ntrain = Ndata - Ntest

    # The training set test set split is saved and loaded
    argTrain = np.random.permutation(Ndata)
    np.savez_compressed(data_dir + '/eventData/combined/argTrain_' + fileLabel + otherLabel + '.npz', argTrain)
    argTrain = loadnpz(data_dir + '/eventData/combined/argTrain_' + fileLabel + otherLabel + '.npz')

    # The data is split into a training set and test set
    argTest = argTrain[:Ntest]
    X_test = torch.tensor(X[argTest]).float()
    Y_test = Y[argTest]
    dataKeys_test = dataKeys[argTest]
#     location_test = torch.tensor(sensorLocation[argTest]).float()
    argTrain = argTrain[Ntest:]
    X = torch.tensor(X[argTrain]).float()
    Y = torch.tensor(Y[argTrain]).long()
    dataKeys = dataKeys[argTrain]
#     location = torch.tensor(sensorLocation[argTrain]).float()

    timeInt = int(len(XtimeInt)/10)
    numEvents = int(len(np.unique(Y)))
    # Load the model
#     model = torch.load('./data/neuronModel/model_16.pt')
    model = SimpleNet3(codeSize, wireName, timeInt, numEvents)

    # The "codeModel" gives the representation for each data set (each session typically. For each rat potentially)
#     codeModel = torch.load(data_dir + '/neuronModel/codeModel_2.pt')
    codeModel = ConstantModel(numDataSet, codeSize)

    # The maximum number of training iterations
    numIter = 100

    # The starting learning rate for the model
    learningRateModel = 0.001
#     learningRateModel = 0.004
    optimizer = torch.optim.Adam(model.parameters(), lr = learningRateModel)

    # The startinging learning rate for the representations
    learningRateCode = 0.002
#     learningRateCode = 0.01
    optimizer2 = torch.optim.SGD(codeModel.parameters(), lr = learningRateCode)

    train_acc = []
    test_losses = []

    done = False # Done will be true once training converges
    iter = -1
    #for iter in range(numIter):
    while (iter < numIter) and (done == False):
        iter += 1
        print (iter)

        batchPerIter = Ntrain // batchSize # The number of batches
        losses = []

        if len(test_losses) > 10:
            maxBefore = np.max(np.array(test_losses)[:-5]) # maximum accuracy prior to the last 5 iterations
            maxNow = np.max(np.array(test_losses)[-5:]) # maximum accuracy in the last 5 iterations

            # If the representation learning rate is down to 0.002 and it converges, then training is done
            if learningRateCode  == 0.002:
                if maxNow < maxBefore:
                    done = True

            # If the representation learning rate is 0.01 and it converges, then the learning rate is reduced.
            if learningRateCode  == 0.01:

                if maxNow < maxBefore + 0.002:
                    print ("Switch Learning Rate")

                    learningRateModel = 0.001
                    optimizer = torch.optim.Adam(model.parameters(), lr = learningRateModel)

                    learningRateCode = 0.002
                    optimizer2 = torch.optim.SGD(codeModel.parameters(), lr = learningRateCode)

        for a in range(batchPerIter):
            argsBatch = (a * batchSize) + np.arange(batchSize)

            # The data of this batch is selected.
            dataKeysBatch = dataKeys[argsBatch]
            XBatch = X[argsBatch]
            YBatch = Y[argsBatch]
#             locBatch = sensorLocation[dataKeysBatch]

            # Commented is an alternative way of disabling representations
            if disableCodes:
                codes = torch.FloatTensor(np.zeros((batchSize, codeSize)))
            else:
                codes = codeModel(dataKeysBatch) 
                codeNoise = torch.FloatTensor(np.random.normal(size=codes.shape)) * noiseScale
                codes = codes + codeNoise
                
#             print("codes:", codes.shape)
#             print("XBatch:", XBatch.shape)
            
            # The model gives a prediction logit (log of probability) called "pred". "encode" is just additional information.
            pred, encode = model(XBatch, codes, returnEncode=True)
            pred_np = pred.data.numpy()

            # Determining the classification accuracy.
            pred_np = np.argmax(pred_np, axis=1)
            diff = YBatch.data.numpy() - pred_np
#             diff = diff[YBatch.data.numpy() < 4]
            accuracy_train = np.argwhere(diff == 0).shape[0] / diff.shape[0]

            # Appending the accuracy to a list
            losses.append(np.mean(accuracy_train))

            # Cross entropy loss for classification
            loss = loss_func(pred, YBatch)
#             loss1 = loss.data.numpy()

            #This gives the list of all representations for all data sets.
            codes = codeModel(np.arange(numDataSet))

            # This adds regularization to the data set representations
            lossCode = torch.mean(codes ** 2)
#             lossCode = torch.mean(torch.mean(codes ** 2, axis=0) ** 0.5) #learningRateCode = 0.002
            loss = loss + (lossCode * codeNorm)

            #l1_lambda = 1e-6
            #l1_lambda = 1e-4
            #l1_lambda = 1e-3
            #l1_lambda = 1e-2
            #l1_lambda = 1e-1
            #l1_reg = torch.tensor(0.)
            #for param in model.parameters():
            #    if len(param.shape) == 2:
            #        #if param.shape[0] == 1680:
            #        if param.shape[0] == 7:
            #            l1_reg += torch.sum(torch.abs(param))
            #loss += l1_lambda * l1_reg
            #loss = loss + (torch.mean(codes ** 2) * codeNorm)

            #Optimizing the model
            optimizer.zero_grad()
            optimizer2.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer2.step()

#         print (np.mean(np.array(losses)))

        # Saving the model in each iteration
        torch.save(model, data_dir + '/neuronModel/model_' + fileLabel + otherLabel + '.pt')
        torch.save(codeModel, data_dir + '/neuronModel/codeModel_' + fileLabel + otherLabel + '.pt')

        if True:
            #X_test = torch.tensor(X[argTest]).float()
            #Y_test = torch.tensor(Y[argTest]).long()
            #dataKeys_test = dataKeys[argTest]

            if True:
                codes = codeModel(dataKeys_test)

                if False: #Enabling this causes noise to be added to representations during testing, just like training. It reduces accuracy.
                    codeNoise = torch.FloatTensor(np.random.normal(size=codes.shape)) * noiseScale
                    codes = codes + codeNoise

            # This gives the models prediction
            pred = model(X_test, codes)
            pred = pred.data.numpy()
            pred = pred[:, :numEvents]

            pred_test = np.copy(pred)
            
            print ("Train Acc", np.mean(np.array(losses)))
            train_acc.append(np.mean(np.array(losses))) # train accuracy

            pred = np.argmax(pred, axis=1)
            diff = Y_test - pred
            accuracy_test = np.argwhere(diff == 0).shape[0] / diff.shape[0]
            
            print (accuracy_test)
            test_losses.append(accuracy_test) # test accuracy

            # Calculating Confusion Matrix
            confusion_matrix = np.zeros((pred.shape[0], numEvents, numEvents))
            confusion_matrix[np.arange(pred.shape[0]),  pred.astype(int), Y_test.astype(int) ] = 1
            confusion_matrix = np.sum(confusion_matrix, axis=0)
            confusion_matrix = confusion_matrix.astype(int)

#           print ("con:",confusion_matrix)

            # Calculating the accuracy of each session (or each rat if rat representations are used)
            accuracySession = []
            for b in range(numDataSet):
                args1 = np.argwhere(dataKeys_test == b)[:, 0]
                
                if args1.shape[0] != 0: # keep getting errors of dividing by 0
                    acc_ses = np.argwhere(diff[args1] == 0).shape[0] / args1.shape[0]
                else:
                    acc_ses = None
                    
                accuracySession.append(acc_ses)

            # Saving accuracy
            np.save(data_dir + '/neuronModel/accuracy_' + fileLabel + otherLabel + '.npy', accuracySession)

    # Saving confusion matrix
    np.save(data_dir + '/neuronModel/cm_' + fileLabel + otherLabel + '.npy', confusion_matrix)
    #Saving the prediction
    np.savez_compressed(data_dir + '/neuronModel/prediction_' + fileLabel + otherLabel + '.npz', pred_test)
    
    # Saving train and test accuracy over each iteration
    np.save(data_dir + '/neuronModel/trainAccuracy_' + fileLabel + otherLabel + '.npy', train_acc)
    np.save(data_dir + '/neuronModel/testAccuracy_' + fileLabel + otherLabel + '.npy', test_losses)
    
    
    

In [8]:
eventNames = loadnpz(data_dir+'/eventData/seperate/outputNames_exp_org.npz')
fileLabel = 'exp_org' # 'S|M & S|NM(NS)'
t = np.arange(100)
timeLabel = '-1to1' 
XtimeInt = t[40:-40]
otherLabel = "" + timeLabel

trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = False)
trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = True)

# codeSize = [20, 40, 60]
# codeNorm = [50, 100]

# for i in codeSize:
#     for j in codeNorm:
#         otherLabel = timeLabel + "cS" + str(i) + "cD" + str(j)
#         print(otherLabel)
#         trainModel(data_dir, fileLabel, otherLabel, XtimeInt, i, j, Rat_rep = False)

0
Train Acc 0.3270336679213718
0.37505227938101215
1
Train Acc 0.3882266833960686
0.402603513174404
2
Train Acc 0.4275538477624425
0.4350167294019239
3
Train Acc 0.46612296110414053
0.4682664157256378
4
Train Acc 0.5027315976578838
0.49477206189878714
5
Train Acc 0.5352232329569218
0.5173567544960268
6
Train Acc 0.5631142827268926
0.5402028439983271
7
Train Acc 0.5874241948975324
0.5575595984943539
8
Train Acc 0.6075909661229612
0.5716227519866165
9
Train Acc 0.6238237139272271
0.5827582601421999
10
Train Acc 0.6385534295273944
0.5933186951066499
11
Train Acc 0.6504861982434128
0.6029381012128816
12
Train Acc 0.6597919280635717
0.6107277289836889
13
Train Acc 0.6699471978251778
0.6203994144709327
14
Train Acc 0.6768742158092849
0.6273002927645337
15
Train Acc 0.6847945420326224
0.6341488916771225
16
Train Acc 0.6921397950648265
0.6390108741112506
17
Train Acc 0.6966750313676286
0.6459117524048515
18
Train Acc 0.7015239439565035
0.651401087411125
19
Train Acc 0.7039157256378085
0.653126

In [9]:
t = np.arange(100)
timeLabel = '-2to2' 
XtimeInt = t[30:-30]
otherLabel = "" + timeLabel

trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = False)
trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = True)

0
Train Acc 0.36741948975324135
0.4290046005855291
1
Train Acc 0.45009933082392306
0.46669803429527396
2
Train Acc 0.48784504391468003
0.49670639899623586
3
Train Acc 0.5216175240485152
0.5266102049351735
4
Train Acc 0.552749895441238
0.5534818067754078
5
Train Acc 0.5825360727728983
0.5759619406106232
6
Train Acc 0.6064408197406943
0.5945734002509411
7
Train Acc 0.6273133626097867
0.6105708908406524
8
Train Acc 0.644500209117524
0.6232225010455876
9
Train Acc 0.6607721664575491
0.6362923462986199
10
Train Acc 0.6740119196988708
0.6468005018820577
11
Train Acc 0.6833176495190297
0.6541718946047679
12
Train Acc 0.693172312839816
0.6604454203262233
13
Train Acc 0.6998117942283564
0.6673462986198243
14
Train Acc 0.7067649519029695
0.6702739439565035
15
Train Acc 0.7125941028858218
0.6760246758678378
16
Train Acc 0.7181095775826014
0.6803638644918444
17
Train Acc 0.7229976997072355
0.6863237139272271
18
Train Acc 0.7265134880803011
0.6909242994562944
19
Train Acc 0.7312839815976578
0.69233

In [10]:
t = np.arange(100)
timeLabel = '-3to1' 
XtimeInt = t[20:-40]
otherLabel = "" + timeLabel

trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = False)
trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = True)

0
Train Acc 0.3428220409870347
0.3752091175240485
1
Train Acc 0.40554422835633624
0.41426181514010874
2
Train Acc 0.4356702216645755
0.44055834378920955
3
Train Acc 0.4649074654956085
0.46329987452948557
4
Train Acc 0.494706712672522
0.4913216227519866
5
Train Acc 0.5282439355918026
0.5208594730238394
6
Train Acc 0.5579255541614387
0.5486198243412798
7
Train Acc 0.5845096194061062
0.5690610623170221
8
Train Acc 0.6058265370138017
0.5868883312421581
9
Train Acc 0.6232094312003346
0.6007946465913844
10
Train Acc 0.6412196779590129
0.6114596403178586
11
Train Acc 0.6544202216645755
0.6194061062317022
12
Train Acc 0.6671241112505228
0.6292346298619824
13
Train Acc 0.676913425345044
0.6377561689669594
14
Train Acc 0.6856440819740695
0.6447093266415725
15
Train Acc 0.6929631953157674
0.652185278126307
16
Train Acc 0.7007005437055626
0.6580928481806776
17
Train Acc 0.7059807611877875
0.6635821831869511
18
Train Acc 0.7100716227519867
0.667607695524885
19
Train Acc 0.716332078628189
0.67351526

Train Acc 0.7530191342534505
0.7258469259723965
62
Train Acc 0.7530583437892095
0.7267356754496027
63
Train Acc 0.7513723337515684
0.7280426599749059
64
Train Acc 0.752849226265161
0.7261083228774571
65
Train Acc 0.7532936010037641
0.7272061898787119
66
Train Acc 0.7546790046005856
0.7284608950230029
67
Train Acc 0.7553455667084902
0.7268925135926391
68
Train Acc 0.7558422208281054
0.7267356754496027
69
Train Acc 0.7555416143872856
0.7308134671685487
70
Train Acc 0.75607747804266
0.730499790882476
71
Train Acc 0.7556461731493099
0.7301338352153911
72
Train Acc 0.7566264115432874
0.7314930991217063
73
Train Acc 0.757319113341698
0.7302906733584275
74
Train Acc 0.7563519447929736
0.7299247176913425
75
Train Acc 0.7588090757005437
0.7306043496445002
76
Train Acc 0.7582078628189042
0.731702216645755
77
Train Acc 0.7596455457967378
0.732120451693852
78
Train Acc 0.7598677331660393
0.73379339188624
79
Train Acc 0.7589267043078209
0.732538686741949
80
Train Acc 0.7590704726056043
0.7316499372

In [11]:
t = np.arange(100)
timeLabel = '-1to3' 
XtimeInt = t[40:-20]
otherLabel = "" + timeLabel

trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = False)
trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = True)

0
Train Acc 0.34601108322877455
0.3951798410706817
1
Train Acc 0.4173332287745713
0.4324027603513174
2
Train Acc 0.45135403596821416
0.46251568381430364
3
Train Acc 0.4876620660811376
0.4992680886658302
4
Train Acc 0.5266102049351735
0.534086156419908
5
Train Acc 0.5609316185696361
0.5577687160184024
6
Train Acc 0.5910576118778753
0.5812421580928482
7
Train Acc 0.6164261815140109
0.6002195734002509
8
Train Acc 0.6381482643245504
0.6177854454203262
9
Train Acc 0.6551390631534922
0.6333647009619406
10
Train Acc 0.6684964450020912
0.6422521957340025
11
Train Acc 0.680586051861146
0.6506691760769553
12
Train Acc 0.6901923881221247
0.6603931409452112
13
Train Acc 0.698478670012547
0.6677122542869093
14
Train Acc 0.7055886658301965
0.6771748222501046
15
Train Acc 0.7123980552070264
0.681409452112087
16
Train Acc 0.7193512128816395
0.6870556252613969
17
Train Acc 0.7250888749477207
0.6937996654119615
18
Train Acc 0.7289575491426181
0.6981388540359682
19
Train Acc 0.7326040359682141
0.70263488

In [12]:
t = np.arange(100)
timeLabel = '-3to3' 
XtimeInt = t[20:-20]
otherLabel = "" + timeLabel

trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = False)
trainModel(data_dir, fileLabel, otherLabel, XtimeInt, Rat_rep = True)

0
Train Acc 0.36151191969887075
0.41656210790464243
1
Train Acc 0.4340103513174404
0.4570263488080301
2
Train Acc 0.47714084065244666
0.4840025094102886
3
Train Acc 0.5135403596821414
0.5135926390631534
4
Train Acc 0.5474043287327478
0.5451693851944793
5
Train Acc 0.5816342534504392
0.576693851944793
6
Train Acc 0.6115119196988708
0.6057089084065245
7
Train Acc 0.6391677122542869
0.6260978670012547
8
Train Acc 0.6591253659556671
0.6458071936428272
9
Train Acc 0.6768219364282727
0.6612296110414053
10
Train Acc 0.6913556043496445
0.6692806357172731
11
Train Acc 0.7034321413634462
0.6807820995399414
12
Train Acc 0.7154563989962359
0.6893559180259305
13
Train Acc 0.7234159347553325
0.697668339606859
14
Train Acc 0.7300423462986199
0.7043078209953995
15
Train Acc 0.7369170849017148
0.7101631116687579
16
Train Acc 0.7423280008364701
0.7143454621497282
17
Train Acc 0.7493465077373483
0.7171685487243831
18
Train Acc 0.7531629025512337
0.7222396486825596
19
Train Acc 0.7576981388540359
0.727101

In [7]:
# eventNames = ['A_MATCH', 'A_NONMATCH', 'B_MATCH', 'B_NONMATCH', 'A_SAMPLES', 'B_SAMPLES']
# eventNames = np.array(eventNames)
# fileLabel_dict = {
# #                   "AM&S": [0,4], 
# #                   "ANM&S": [1,4], 
# #                   "BM&S": [2,5], 
# #                   "BNM&S": [3,5], 
# #                   "AM&NM&S": [0,1,4], 
#                   "BM&NM&S": [2,3,5]
#                  }

# # X = X[:, :, :] # -5 to 5
# # X = X[:, :, 20:] # -3 to 5
# # X = X[:, :, :-20] # -5 to 3
# # X = X[:, :, 20:-20] # -3 to 3
# # X = X[:, :, 20:-40] # -3 to 1
# # X = X[:, :, 40:-20] # -1 to 3
# # X = X[:, :, 30:-30] # -2 to 2
# # X = X[:, :, 35:-35] # -1.5 to 1.5
# # X = X[:, :, 40:-40] # -1 to 1
# # X = X[:, :, np.concatenate((np.arange(40), np.arange(40)+60))] # -5 to -1 & 1 to 5 (non-center)
# # X = X[:, :, 60:] # 1 to 5 (non-center)

# t = np.arange(100)
# XtimeInt_dict = {
# #                 "-5to5": t, 
#                 "-3to5": t[20:], 
#                 "-5to3": t[:-20], 
# #                 "-3to3": t[20:-20], 
#                 "-3to1": t[20:-40], 
#                 "-1to3": t[40:-20], 
# #                 "-2to2": t[30:-30],
# #                 "-1.5to1.5": t[35:-35],
# #                 "-1to1": t[40:-40]
#                 } #, np.concatenate((np.arange(40), np.arange(40)+60)), t[60:]]
    
# count = 0

# for i in fileLabel_dict:
#     print(i, fileLabel_dict[i]) # i = fileLabel
#     print("Included Events",eventNames[fileLabel_dict[i]]) # eventNames
    
#     for j in XtimeInt_dict:
#         print("Time Interval is: ", j, len(XtimeInt_dict[j]))
#         trainModel(data_dir, i, j, XtimeInt_dict[j], Rat_rep = False)
#         count+=1
#         print("Complete " + i + j + ", Session Representation" + f"({count})")

#         trainModel(data_dir, i, j, XtimeInt_dict[j], Rat_rep = True)
#         count+=1
#         print("Complete " + i + j + ", Rat Representation" +f"({count})")
